In [1]:
import os
os.path.split("x/y/z.txt")

('x/y', 'z.txt')

In [3]:
pwd

'/config/workspace/research'

In [6]:
pwd

'/Users/shubhamkumar/DEEPCNNCLASSIFIER/research'

In [4]:
cd ..

/config/workspace


In [5]:
os.path.getsize("template.py")

1904

In [1]:
from box import ConfigBox

In [3]:
d = {"key": "value", "key1" : "value1"}
d["key"]

'value'

In [4]:
d.key

AttributeError: 'dict' object has no attribute 'key'

In [6]:
# configbox will help us to get the value of d.keys

In [7]:
d2 = ConfigBox({"key": "value", "key1" : "value1"})
d2

ConfigBox({'key': 'value', 'key1': 'value1'})

In [8]:
d2.key

'value'

In [19]:
from ensure import ensure_annotations

In [20]:
@ensure_annotations
def get_product(x: int, y:int) -> int:
    return x * y

In [21]:
get_product(x = 'hi', y = 7)

EnsureError: Argument x of type <class 'str'> to <function get_product at 0x7f064b269dc0> does not match annotation type <class 'int'>

In [24]:
#Decorators give some extra functionalities to our existing function. 
# Here we have the control over input and output.

In [25]:
@ensure_annotations
def get_product(x: int, y:int) -> str:
    return x * y

In [26]:
get_product(x = 4, y = 9)

EnsureError: Return value of <function get_product at 0x7f064b160820> of type <class 'int'> does not match annotation type <class 'str'>

In [28]:
@ensure_annotations
def get_product(x: int, y:int) -> str:
    return str(x * y)

In [29]:
get_product(x = 4, y = 9)

'36'

### CREATING THE WORKFLOW FOR DATA INGESTION

In [69]:
pwd

'/config/workspace'

In [42]:
from collections import namedtuple
import os
os.chdir("../")

In [60]:
DataIngestionConfig = namedtuple("DataIngestionCofig", [
"root_dir",
"source_URL",
"local_data_file",
"unzip_dir"
])

WE CAN ALSO USE DATACLASSES IN PLACE OF NAMEDTUPLES

In [70]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    #We define class variables here
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

DEFINING THE CONFIGURATION

In [71]:
from DeepClassifier.constants import *
from DeepClassifier.utils import read_yaml, create_directories
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])
        data_ingestion_config = DataIngestionConfig(
            root_dir = config.root_dir,
            source_URL = config.source_URL,
            local_data_file = config.local_data_file,
            unzip_dir = config.unzip_dir
        )

        return data_ingestion_config

DEFINING THE COMPONENTS

In [72]:
import os
import urllib.request as request
from zipfile import ZipFile

class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )

    def _get_updated_list_of_files(self, list_of_files):
        return [f for f in list_of_files if f.endswith(".jpg") and ("Cat" in f or "Dog" in f)]

    def _preprocess(self, zf: ZipFile, f: str, working_dir: str):
        target_filepath = os.path.join(working_dir, f)
        if not os.path.exists(target_filepath):
            zf.extract(f, working_dir)

        if os.path.getsize(target_filepath) == 0:
            os.remove(target_filepath)
    def unzip_and_clean(self):
        with ZipFile(file=self.config.local_data_file, mode="r") as zf:
            list_of_files = zf.namelist()
            updated_list_of_files = self._get_updated_list_of_files(list_of_files)
            for f in updated_list_of_files:
                self._preprocess(zf, f, self.config.unzip_dir)

In [73]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.unzip_and_clean()
except Exception as e:
    raise e

[2022-12-20 15:09:26,145: INFO: common]: yaml file: configs/config.yaml loaded successfully
[2022-12-20 15:09:26,147: INFO: common]: yaml file: params.yaml loaded successfully
[2022-12-20 15:09:26,149: INFO: common]: created directory at: artifacts
[2022-12-20 15:09:26,150: INFO: common]: created directory at: artifacts/data_ingestion


In [1]:
x = 2

In [3]:
assert x == 2

In [4]:
assert x != 2

AssertionError: 